Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fdouble-vs-another-attempt-207994%2Fdouble_vs_another_attempt.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/double-vs-another-attempt-207994)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Over time, how does doubling dice on a crit compare vs having another action?”](https://rpg.stackexchange.com/a/208013/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H
from dyce.evaluation import HResult, expandable
from enum import IntEnum
from fractions import Fraction

class HitOutcome(IntEnum):
    CRIT_MISS = -1
    MISS = 0
    HIT = 1
    CRIT_HIT = 2

d20 = H(20)
MIN_DMG = 0  # See <https://www.dndbeyond.com/sources/basic-rules/combat#DamageRolls>

def hit_outcomes_from_target_bonus(
    target: int,
    hit_bonus: int,
    crit_misses: bool = True,
) -> H:
    def _outcome_count_gen():
        for outcome, count in d20.items():
            if outcome == max(d20):
                yield HitOutcome.CRIT_HIT, count
            elif crit_misses and outcome == min(d20):
                yield HitOutcome.CRIT_MISS, count
            elif outcome + hit_bonus >= target:
                yield HitOutcome.HIT, count
            else:
                yield HitOutcome.MISS, count

    return H(_outcome_count_gen())

def double_dice(dmg_die: H, dmg_bonus: int) -> H:
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in (2 @ dmg_die + dmg_bonus).items()
    )

def double_dmg_house_rule(dmg_die: H, dmg_bonus: int) -> H:
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in ((dmg_die + dmg_bonus) * 2).items()
    )

def another_bite(
    dmg_die: H,
    target: int,
    hit_bonus: int,
    dmg_bonus: int,
    crit_misses_on_rerolls: bool = True,
    dmg_bonus_on_rerolls: bool = False,
) -> H:
    @expandable
    def _explode_crit_hits(result: HResult):
        reroll_dmg_bonus = dmg_bonus if dmg_bonus_on_rerolls else 0
        if result.outcome is HitOutcome.CRIT_HIT:
            return dmg_die + reroll_dmg_bonus + _explode_crit_hits(result.h)  # result.h is hit_outcomes
        elif result.outcome is HitOutcome.HIT:
            return dmg_die + reroll_dmg_bonus
        else:  # treats HitOutcome.MISS and HitOutcome.CRIT_MISS as equivalent
            return H({0: 1})  # no (more) damage

    hit_outcomes = hit_outcomes_from_target_bonus(target, hit_bonus, crit_misses_on_rerolls)
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in
        (dmg_die + _explode_crit_hits(hit_outcomes, limit=Fraction(1, 10_000)) + dmg_bonus).items()
    )

In [3]:
NUM_COLS=4
DMG_DICE = {
    "d4": H(4),
    "d6": H(6),
    "d8": H(8),
    "d10": H(10),
    "d12": H(12),
    "d20": H(20),
    "2d4": 2 @ H(4),
    "2d6": 2 @ H(6),
    "2d8": 2 @ H(8),
    # ...
}
SELECTED_DMG_DICE = list(DMG_DICE)[:5]  # start by selecting first five values (whatever they are)

In [4]:
# Interactive UI code
from anydyce import HPlotterChooser
from anydyce.viz import PlotWidgets
from IPython.display import display
from ipywidgets import widgets

def _display(
    selected_dmg_dice: list[str],
    target_range: tuple[int, int],
    hit_bonus: int,
    dmg_bonus: int,
    crit_misses_on_rerolls: bool,
    dmg_bonus_on_rerolls: bool,
) -> None:
    results = []
    target_lo, target_hi = target_range
    target_hi += 1  # exclusive upper bound
    num_charts = 1 + (target_hi - target_lo)  # one for double dmg plus one for each target
    fillers = -num_charts % NUM_COLS
    
    for dmg_die_name in selected_dmg_dice:
        dmg_die = DMG_DICE[dmg_die_name]
        double_dice_res = double_dice(dmg_die, dmg_bonus)
        double_dmg_res = double_dmg_house_rule(dmg_die, dmg_bonus)
        results.append(
            (
                f"{dmg_die_name} double dice ({double_dice_res.mean():0.3f}) vs.\ndouble all damage ({double_dmg_res.mean():0.3f})",
                double_dice_res,
                double_dmg_res,
            )
        )
        for target in range(target_lo, target_hi):
            another_bite_res = another_bite(dmg_die, target, hit_bonus, dmg_bonus, crit_misses_on_rerolls, dmg_bonus_on_rerolls)
            results.append(
                (
                    f"{dmg_die_name} double dice ({double_dice_res.mean():0.3f}) vs.\nanother bite @AC{target} ({another_bite_res.mean():0.3f})",
                    double_dice_res,
                    another_bite_res,
                )
            )
        results.extend((f"", H({})) for _ in range(fillers))
    chooser.update_hs(results)

dmg_dice_widget = widgets.SelectMultiple(
    options=list(DMG_DICE),
    value=SELECTED_DMG_DICE,
    description="Dmg Dice",
)
target_range_widget = widgets.IntRangeSlider(
    value=(13, 19),
    min=2,
    max=30,
    step=1,
    continuous_update=False,
    description="AC Range",
)
hit_bonus_widget = widgets.BoundedIntText(
    value=11,
    min=-10,
    max=20,
    step=1,
    description='Hit Bonus',
)
dmg_bonus_widget = widgets.BoundedIntText(
    value=5,
    min=-10,
    max=20,
    step=1,
    description='Dmg Bonus',
)
dmg_bonus_widget = widgets.BoundedIntText(
    value=5,
    min=-10,
    max=20,
    step=1,
    description='Dmg Bonus',
)
crit_misses_on_rerolls_widget = widgets.Checkbox(value=True, description='Crit Misses Each Roll')
dmg_bonus_on_rerolls_widget = widgets.Checkbox(value=False, description='Dmg Bonus Each Roll')
chooser = HPlotterChooser(
    plot_widgets=PlotWidgets(
        initial_burst_columns=NUM_COLS,
        initial_burst_zero_fill_normalize=True,
        initial_resolution=18,
    )
)

display(
    widgets.HBox([
        widgets.VBox([dmg_dice_widget, target_range_widget]),
        widgets.VBox([
            hit_bonus_widget,
            dmg_bonus_widget,
            crit_misses_on_rerolls_widget,
            dmg_bonus_on_rerolls_widget,
        ]),
    ]),
    widgets.interactive_output(
        _display,
        {
            "selected_dmg_dice": dmg_dice_widget,
            "target_range": target_range_widget,
            "hit_bonus": hit_bonus_widget,
            "dmg_bonus": dmg_bonus_widget,
            "crit_misses_on_rerolls": crit_misses_on_rerolls_widget,
            "dmg_bonus_on_rerolls": dmg_bonus_on_rerolls_widget,
        },
    ),
)

chooser.interact()

Output()